In [1]:
import numpy as np
import airo_models
from pydrake.common import temp_directory
from pydrake.geometry import StartMeshcat
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.analysis import Simulator
from pydrake.geometry import MeshcatVisualizer
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker
from tools.visualization import add_meshcat_triad
from tools.scenes import *
from tools.building import *
from typing import Optional, Tuple

from pydrake.geometry import Meshcat, MeshcatVisualizer, MeshcatVisualizerParams, Role
from pydrake.planning import RobotDiagram, RobotDiagramBuilder
from pydrake.systems.framework import Context
from pydrake.visualization import ApplyVisualizationConfig, VisualizationConfig

In [2]:
#
# Setup initial robot diagram
#

robot_diagram_builder = RobotDiagramBuilder()
scene_graph = robot_diagram_builder.scene_graph()
builder = robot_diagram_builder.builder()

# Adding Meshcat must also be done before finalizing
meshcat = Meshcat()
meshcat.SetCameraPose([-2.0, 0, 1.0], [0, 0, 0])
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

# Add visualizer for proximity/collision geometry
collision_params = MeshcatVisualizerParams(role=Role.kProximity, prefix="collision", visible_by_default=False)
MeshcatVisualizer.AddToBuilder(builder, scene_graph.get_query_output_port(), meshcat, collision_params)

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [3]:
#
# Setup the plant
#

meshcat

plant = robot_diagram_builder.plant()
parser = robot_diagram_builder.parser()
parser.SetAutoRenaming(True)

# Load URDF files
ur5e_urdf_path = airo_models.get_urdf_path("ur3e")
probe_tool_path = "./tools/urdf/probe_tool.urdf"

table_thickness = 0.2
table_urdf_path = airo_models.box_urdf_path((2.0, 2.4, table_thickness), "table")

arm_index = parser.AddModels(ur5e_urdf_path)[0]
gripper_index = parser.AddModels(probe_tool_path)[0]
table_index = parser.AddModels(table_urdf_path)[0]

# Weld some frames together
world_frame = plant.world_frame()
table_frame = plant.GetFrameByName("base_link", table_index)
arm_frame = plant.GetFrameByName("base_link", arm_index)
arm_tool_frame = plant.GetFrameByName("tool0", arm_index)
gripper_frame = plant.GetFrameByName("base_link", gripper_index)

arm_transform = RigidTransform(rpy=RollPitchYaw([0, 0, np.pi]), p=[0, 0, 0])
table_transform = RigidTransform(p=[0, 0, -table_thickness / 2])

plant.WeldFrames(world_frame, arm_frame, arm_transform)
plant.WeldFrames(arm_tool_frame, gripper_frame, X_URTOOL0_ROBOTIQ)
plant.WeldFrames(world_frame, table_frame, table_transform)

Failed to load material file(s). Use default material.
material [ 'probeMaterial0' ] not found in .mtl



<WeldJoint name='world_welds_to_base_link' index=12 model_instance=4>

In [4]:
#
# Finalize build
#

diagram, context = finish_build(robot_diagram_builder, meshcat)
plant = diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

collision_checker = SceneGraphCollisionChecker(
    model=diagram,
    robot_model_instances=[arm_index, gripper_index],
    edge_step_size=0.125,  # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding=0.005,
    self_collision_padding=0.005,
)

INFO:drake:Allocating contexts to support implicit context parallelism 12


In [5]:
#
# Setup initial pose and planner
#

start_joints = np.deg2rad([0, -90, -90, -90, 90, 0])
plant.SetPositions(plant_context, arm_index, start_joints)
diagram.ForcedPublish(context)

add_meshcat_triad(meshcat, "World", X_W_Triad=RigidTransform(p=[0, 0, 0], rpy=RollPitchYaw([0, 0, 0])))

# Planner setup
from ur_analytic_ik import ur3e
from tools.ompl.single_arm_planner import SingleArmOmplPlanner
from tools.visualization import publish_joint_path

tcp_transform = np.identity(4)

def inverse_kinematics_fn(tcp_pose):
    solutions_1x6 = ur3e.inverse_kinematics_with_tcp(tcp_pose, tcp_transform)
    solutions = [solution.squeeze() for solution in solutions_1x6]
    return solutions

planner = SingleArmOmplPlanner(collision_checker.CheckConfigCollisionFree, inverse_kinematics_fn)

ModuleNotFoundError: No module named 'ompl'

In [ ]:
#
# Class used to interface with robot
# 

class RobotInterface

    def __init__(self):
        self.tf_offset = [0, 0, 0, 0, 0, 0]

    def getTCPPose(self):
        pass

    def getTFValue(self):
        pass

    def zeroTFSensor(self):
        self.tf_offset = self.getTFValue()

class DrakeRobot(RobotInterface):


In [ ]:
#
# List of functions to help plan stuff
#

def retract_probe(start_joints, transform, z_height, duration):
    """
    Moves the probe in the z-direction. 
    """

    pos = transform.translation()
    pos[2] += z_height
    
    new_transform = RigidTransform(p=pos, rpy=transform.rotation())
    tcp_pose = np.ascontiguousarray(new_transform.GetAsMatrix4())

    path = planner.plan_to_tcp_pose(start_joints, tcp_pose)

    publish_joint_path(path, duration, meshcat, diagram, context, arm_index)
    add_meshcat_triad(meshcat, "TCP Frame", X_W_Triad=transform)

In [29]:
# Now plan to tcp pose

transform = RigidTransform(p=[0.3, 0, 0.3], rpy=RollPitchYaw([np.pi, 0, 0]))
tcp_pose_0 = np.ascontiguousarray(transform.GetAsMatrix4())

path = planner.plan_to_tcp_pose(start_joints, tcp_pose_0)

publish_joint_path(path, 5.0, meshcat, diagram, context, arm_index)
add_meshcat_triad(meshcat, "TCP Frame", X_W_Triad=transform)

2024-02-26 00:17:08.284 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:156 - IK returned 8 solutions.
2024-02-26 00:17:08.285 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:167 - Found 8/8 solutions within joint bounds.
2024-02-26 00:17:08.291 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:174 - Found 4/8 valid solutions.
2024-02-26 00:17:09.118 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:203 - Found 4 paths towards IK solutions:
2024-02-26 00:17:09.118 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:204 - Path lengths: [5.072, 5.665, 1.26, 6.502]
2024-02-26 00:17:09.119 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:205 - Path distances: [5.072, 5.665, 1.26, 4.875]
2024-02-26 00:17:09.120 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:220 - Length of chosen solution (= shortest path): 1.260


Debug:   RRTConnect: Planner range detected to be 6.156239
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 5 states (2 start + 3 goal)
Info:    Solution found in 0.074511 seconds
Info:    SimpleSetup: Path simplification took 0.099209 seconds and changed from 3 to 2 states
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 4 states (2 start + 2 goal)
Info:    Solution found in 0.023282 seconds
Info:    SimpleSetup: Path simplification took 0.110146 seconds and changed from 3 to 2 states
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 5 states (2 start + 3 goal)
Info:    Solution found in 0.018189 seconds
Info:    SimpleSetup: Path simplification took 0.048382 seconds and changed from 4 to 2 states
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 7 states (4 